<a href="https://colab.research.google.com/github/simsang1l/HG-ML-DL/blob/main/05/hg_05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.3, random_state = 42)

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

In [ ]:
print(sub_input.shape, val_input.shape)

(3637, 3) (910, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9980753368160572
0.8428571428571429


In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.02483773, 0.03289771, 0.02143764, 0.00917625, 0.00900364]), 'score_time': array([0.00091982, 0.00102258, 0.00104952, 0.00089216, 0.00085235]), 'test_score': array([0.84615385, 0.85494505, 0.87128713, 0.85148515, 0.83938394])}


In [ ]:
import numpy as np

print(np.mean(scores['test_score']))

0.8526510233440927


In [ ]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.8711219913200111


In [ ]:
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8709004211647382


In [ ]:
from sklearn.model_selection import GridSearchCV
params = {"min_impurity_decrease" : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state= 42), params, n_jobs = -1)

gs.fit(train_input, train_target)
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9133494611831977


In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0003}


In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.85814795 0.86892371 0.87112199 0.86870296 0.86540239]


In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0003}


In [ ]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1), 
          'min_samples_split': range(2, 100, 10)
          }

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

print(gs.best_params_)

{'max_depth': 13, 'min_impurity_decrease': 0.00030000000000000003, 'min_samples_split': 2}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8722221013310122


In [ ]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10 )

array([7, 5, 2, 2, 3, 6, 3, 5, 0, 8])

In [ ]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 98,  92,  94,  91, 121, 110,  98,  94,  90, 112]))

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.44694919, 0.10595153, 0.50672929, 0.35287034, 0.55121168,
       0.26524497, 0.14273173, 0.73842019, 0.49014961, 0.6505493 ])

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs = -1, random_state = 42)
gs.fit(train_input, train_target)

print(gs.best_params_)

{'max_depth': 42, 'min_impurity_decrease': 0.00037864646423661145, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8715627606716716


In [ ]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.861025641025641
